In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torchvision
from torchvision import transforms, datasets
import argparse
import torch.nn as nn
import torch.nn.functional as F
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

from torchvision.transforms import Compose, ToTensor, Resize
import torch.utils.data as data
from os import listdir
from os.path import join
from PIL import Image
from torch.utils.data import DataLoader

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 1, stride=1, padding=0)
        for j in range(3):
            for i in range(6):
                exec("self.Resconv1"+str(j)+str(i)+"="+"nn.Conv2d(32, 32, 3, stride=1, padding=1)")
                exec("self.Batch_norm1"+str(j)+str(i)+"="+"nn.BatchNorm2d(32, track_running_stats=False)")
                exec("self.Resconv2"+str(j)+str(i)+"="+"nn.Conv2d(32, 32, 3, stride=1, padding=1)")
                exec("self.Batch_norm2"+str(j)+str(i)+"="+"nn.BatchNorm2d(32, track_running_stats=False)")
        self.Transconv1 = nn.ConvTranspose2d(32, 32, 3, stride=2, padding=1, output_padding = 1)
        self.Transconv2 = nn.ConvTranspose2d(32, 32, 3, stride=2, padding=1, output_padding = 1)
        self.conv2 = nn.Conv2d(32, 3*256, 1, stride=1, padding=0)

        self.Maskconv1 = nn.Conv2d(3, 64, 7, stride=1, padding=3)
        for i in range(20):
            exec("self.Gateconv1"+str(i)+ "="+ "nn.Conv2d(64, 2*64, 5, stride=1, padding=2)")
            exec("self.Gateconv2"+str(i)+ "="+ "nn.Conv2d(2*64, 2*64, (1,1), stride=1, padding=0)")
            exec("self.Gateconv3"+str(i)+ "="+ "nn.Conv2d(64, 2*64, (1,5), stride=1, padding=(0,2))")
            exec("self.Gateconv4"+str(i)+ "="+ "nn.Conv2d(64, 64, (1,1), stride=1, padding=0)")
        self.Maskconv2 = nn.Conv2d(64, 1024, 1, stride=1, padding=0)
        self.Maskconv3 = nn.Conv2d(1024, 3*256, 1, stride=1, padding=0)

#         self.conv1.weight = nn.Parameter(self.weight_init(3, 32, (1,1), mask_type = None))
#         self.Resconv1.weight = nn.Parameter(self.weight_init(32, 32, (3,3), mask_type = None))
#         self.Transconv1.weight = nn.Parameter(self.weight_init(32, 32, (3,3), mask_type = None))
#         self.Resconv2.weight = nn.Parameter(self.weight_init(32, 32, (3,3), mask_type = None))
#         self.Transconv2.weight = nn.Parameter(self.weight_init(32, 32, (3,3), mask_type = None))
#         self.conv2.weight = nn.Parameter(self.weight_init(32, 3*256, (1,1), mask_type = None))

        self.Maskconv1.weight = nn.Parameter(self.weight_init(3, 64, (7,7), mask_type='A'))
        for i in range(20):
            exec("self.Gateconv1"+str(i)+".weight"+"="+"nn.Parameter(self.weight_init(64, 2*64, (5,5), mask_type='C'))")
            exec("self.Gateconv2"+str(i)+".weight"+"="+"nn.Parameter(self.weight_init(2*64, 2*64, (1,1), mask_type=None))")
            exec("self.Gateconv3"+str(i)+".weight"+"="+ "nn.Parameter(self.weight_init(64, 2*64, (1,5), mask_type='B'))")
            exec("self.Gateconv4"+str(i)+".weight"+"="+"nn.Parameter(self.weight_init(64, 64, (1,1), mask_type='B'))")
        self.Maskconv2.weight = nn.Parameter(self.weight_init(64, 1024, (1,1), mask_type='B'))
        self.Maskconv3.weight = nn.Parameter(self.weight_init(1024, 3*256, (1,1), mask_type='B'))

    def weight_init(self, in_channel, num_outputs, kernel_shape, mask_type=None):
        kernel_h, kernel_w = kernel_shape
        center_h = kernel_h // 2
        center_w = kernel_w // 2

        mask = np.zeros((num_outputs, in_channel, kernel_h, kernel_w), dtype=np.float32)
        if mask_type is not None:
            mask[:, :, :center_h, :] = 1
            if mask_type == 'A':
                mask[:, :, center_h, :center_w] = 1
            if mask_type == 'B':
                mask[:, :, center_h, :center_w+1] = 1
        else:
            mask[:, :, :, :] = 1

        weights_shape = [num_outputs, in_channel, kernel_h, kernel_w]
        weights = torch.empty(weights_shape, requires_grad=True, device="cuda")
        weights = nn.init.xavier_normal_(weights)
        weights = weights*torch.from_numpy(mask).to("cuda")
        return weights

    def prior_network(self, hr_images):
        conv1 = self.Maskconv1(hr_images)
        inputs = conv1
        state = conv1
        for i in range(20):
            inputs, state = self.gated_conv2d(inputs, state, [5, 5], i)
        conv2 = self.Maskconv2(inputs)
        conv2 = F.relu(conv2)
        prior_logits = self.Maskconv3(conv2)
        prior_logits = prior_logits.permute(0, 2, 3, 1)
        prior_logits = torch.cat((prior_logits[:, :, :, 0::3], prior_logits[:, :, :, 1::3], prior_logits[:, :, :, 2::3]), 1)
        prior_logits = torch.permute(0, 3, 1, 2)
        return prior_logits

    def conditioning_network(self, lr_images):
        res_num = 6
        inputs = lr_images
        inputs = self.conv1(inputs)
        for i in range(2):
            for j in range(res_num):
                inputs = self.resnet_block(inputs, i, j)
            inputs = eval("self.Transconv"+str(i+1))(inputs)
            inputs = F.relu(inputs)
        for i in range(res_num):
            inputs = self.resnet_block(inputs, 2, i)
        conditioning_logits = self.conv2(inputs)
        return conditioning_logits

    def resnet_block(self, inputs, i, j):
        conv1 = eval("self.Resconv1"+str(i)+str(j))(inputs)
        bn1 = eval("self.Batch_norm1"+str(i)+str(j))(conv1)
        relu1 = F.relu(bn1)
        conv2 = eval("self.Resconv2"+str(i)+str(j))(relu1)
        bn2 = eval("self.Batch_norm2"+str(i)+str(j))(conv2)
        output = inputs + bn2
        return output


    def gated_conv2d(self, inputs, state, kernel_shape, i):
        batch_size, in_channel, height, width  = list(inputs.size())
        kernel_h, kernel_w = kernel_shape
        left = eval("self.Gateconv1"+str(i))(state)
        left1 = left[:, 0:in_channel, :, :]
        left2 = left[:, in_channel:, :, :]
        left1 = F.tanh(left1)
        left2 = F.sigmoid(left2)
        new_state = left1 * left2
        left2right = eval("self.Gateconv2"+str(i))(left)
        right = eval("self.Gateconv3"+str(i))(inputs)
        right = right + left2right
        right1 = right[:, 0:in_channel, :, :]
        right2 = right[:, in_channel:, :, :]
        right1 = F.tanh(right1)
        right2 = F.sigmoid(right2)
        up_right = right1 * right2
        up_right = eval("self.Gateconv4"+str(i))(up_right)
        outputs = inputs + up_right
        return outputs, new_state

    def forward(self, lr_images, hr_images):
        hr_images = hr_images - 0.5
        lr_images = lr_images - 0.5
        prior_logits = self.prior_network(hr_images)
        conditioning_logits = self.conditioning_network(lr_images)
        return prior_logits, conditioning_logits


In [3]:
def input_transform(crop_size):
    return Compose([Resize((8,8),interpolation = 2),ToTensor(),])
def target_transform(crop_size):
    return Compose([Resize((32,32),interpolation = 2),ToTensor(),])
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])
def load_img(filepath):
    img = Image.open(filepath)
    return img

class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, input_transform=None, target_transform=None):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]

        self.input_transform = input_transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        inputs = load_img(self.image_filenames[index])
        target = inputs.copy()
        inputs = self.input_transform(inputs)
        target = self.target_transform(target)

        return inputs, target

    def __len__(self):
        return len(self.image_filenames)
    
###
def softmax_loss(logits, labels):
    logits = logits.permute(0, 2, 3, 1)
    logits = torch.reshape(logits, [-1, 256])
    labels = labels.to(torch.int64)
    labels = labels.permute(0, 2, 3, 1)
    labels = torch.reshape(labels, [-1])
    return F.cross_entropy(logits, labels)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader,1):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        prior_logits, conditioning_logits = model(lr_images = (data), hr_images = (target))
        l1 = softmax_loss(prior_logits+conditioning_logits, torch.floor(target*255))
        l2 = softmax_loss(conditioning_logits, torch.floor(target*255))
        loss = l1+l2
        loss3 = softmax_loss(prior_logits, torch.floor(target*255))
        loss.backward()
        optimizer.step()
        if batch_idx % 1 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), (len(train_loader)*len(data)),
                100. * batch_idx / len(train_loader), loss.item()))

        if batch_idx % 1000 == 0:
            sample(model, data, target, len(data), mu=1.1, step=epoch*batch_idx)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(train_loader,1):
            data, target = data.to(device), target.to(device)
            prior_logits, conditioning_logits = model(lr_images = (data), hr_images = (target))
            l1 = softmax_loss(prior_logits+conditioning_logits, torch.floor(target*255))
            l2 = softmax_loss(conditioning_logits, torch.floor(target*255))
            test_loss += l1+l2 # sum up batch loss

    test_loss /= len(test_loader)*32
    print("test_loss : ", test_loss.item())

In [4]:
def logits_2_pixel_value(logits, mu=1.1):
    rebalance_logits = logits * mu
    probs = softmax(rebalance_logits)
    pixel_dict = torch.arange(0, 256, dtype=torch.float32).to("cuda")
    pixels = torch.sum(probs*pixel_dict, dim=1)
    return pixels/255

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    a, b = torch.max(x, -1, keepdim=True, out=None)
    e_x = torch.exp(x - a)
    return e_x / e_x.sum(dim=-1, keepdim =True) # only difference

def sample(model, data, target, batch_size, mu=1.1, step=None):
    with torch.no_grad():
        np_lr_imgs = data
        np_hr_imgs = target
        c_logits = model.conditioning_network
        p_logits = model.prior_network
        gen_hr_imgs = torch.zeros((batch_size, 3, 32, 32), dtype=torch.float32).to("cuda")
        np_c_logits = c_logits(torch.floor(np_lr_imgs*255))
#         print("torch.floor(np_lr_imgs*255)", torch.floor(np_lr_imgs*255)[0][:,0,0])
#         print("c_logits", np_c_logits[0].shape, torch.max(np_c_logits[0][:,0,0], -1))
#         print("p_logits", p_logits(gen_hr_imgs)[0].shape, p_logits(gen_hr_imgs)[0][:,0,0])
#         print("target", torch.floor(target*255)[0][:,0,0])
        for i in range(32):
            print(i)
            for j in range(32):
                for c in range(3):
#                     print(i,j,c)
                    np_p_logits = p_logits(gen_hr_imgs)
                    new_pixel = logits_2_pixel_value(np_c_logits[:, c*256:(c+1)*256, i, j] + np_p_logits[:, c*256:(c+1)*256, i, j], mu=mu)
                    gen_hr_imgs[:, c, i, j] = new_pixel
        samples_dir = "/home/eee/ug/15084005/DIH/samples/"
        print("sample")
        save_samples(np_lr_imgs, samples_dir + '/lr_' + str(mu*10) + '_' + str(step))
        save_samples(np_hr_imgs, samples_dir + '/hr_' + str(mu*10) + '_' + str(step))
        save_samples(gen_hr_imgs, samples_dir + '/generate_' + str(mu*10) + '_' + str(step))

def save_samples(np_imgs, img_path):
    print("save")
    torchvision.utils.save_image(np_imgs[0, :, :, :], img_path+".png")


In [5]:
def main():
    use_gpu = True
    num_epoch = 30
    batch_size = 32
    learning_rate = 4e-4

    use_cuda = use_gpu and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.cuda.set_device(0)

    image_dir = '/home/eee/ug/15084005/DIH/CelebA/CelebA/train/img_align_celeba/'
    train_set = DatasetFromFolder(image_dir,input_transform=input_transform(1),target_transform=target_transform(1))
    train_loader = DataLoader(dataset=train_set, batch_size = batch_size, shuffle=True)
    
    image_dir2 = '/home/eee/ug/15084005/DIH/CelebA/CelebA/test/data/'
    test_set = DatasetFromFolder(image_dir2,input_transform=input_transform(1),target_transform=target_transform(1))
    test_loader = DataLoader(dataset=train_set, batch_size = batch_size, shuffle=True)

    model = Net()
    model = model.to(device)

    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.95)
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    for epoch in range(1, num_epoch+1):
        train(model, device, train_loader, optimizer, epoch)
        # sample(model, train_loader_lr, train_loader_hr, mu=1.1, step=epoch)
        test(model, device, test_loader)

In [6]:
main()

RuntimeError: cuda runtime error (46) : all CUDA-capable devices are busy or unavailable at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/THC/THCTensorRandom.cu:25